# DataFed Checklist

In [ ]:
import os
import subprocess
from platform import platform
import sys
import ctypes

try:
    # This package is not part of anaconda and may need to be
    # installed.
    from datafed.CommandLib import API
except ImportError:
    print("datafed not found. Installing from pip.")
    subprocess.call([sys.executable, "-m", "pip", "install", "datafed"])
    from datafed.CommandLib import API

from datafed import version as df_ver

## 0. Machine information:

Python version:

In [ ]:
sys.version_info

In [ ]:
platform()

## 1. Verify DataFed installation:

In [ ]:
if not df_ver.startswith("1.1"):
    print("Attempting to update DataFed.")
    subprocess.call([sys.executable, "-m", "pip", "install", "--upgrade", "datafed"])
    print(
        "Please restart the python kernel or upgrade manually to V "
        " 1.1.0:1 if you are repeatedly seeing this message via"
        "\n\tpip install --upgrade datafed"
    )
else:
    df_api = API()
    print("Success! You have DataFed: " + df_ver)

## 2. Verify user authentication: 

In [ ]:
if df_api.getAuthUser():
    print(
        "Success! You have been authenticated into DataFed as: "
        df_api.getAuthUser()
    )
else:
    print("You have not authenticated into DataFed Client")
    print(
        'Please follow instructions in the "Basic Configuration" '
        'section in the link below to authenticate yourself:'
    )
    print("https://ornl.github.io/DataFed/user/client/"
          "install.html#basic-configuration")

## 3. Ensure Globus Endpoint is set:

In [ ]:
if not df_api.endpointDefaultGet():
    print("Please follow instructions in the link below to find " "your Globus Endpoint:")
    print(
        "https://ornl.github.io/DataFed/system/getting_started."
        "html#install-identify-globus-endpoint"
    )
    endpoint = input(
        "\nPlease enter either the Endpoint UUID or Legacy Name " "for your Globus Endpoint: "
    )
    df_api.endpointDefaultSet(endpoint)

print("Your default Globus Endpoint in DataFed is:\n" f"{df_api.endpointDefaultGet()}")

## 4. Test Globus Endpoint:
And in the case of Windows - write access to the folder:

In [ ]:
dget_resp = df_api.dataGet("d/35437908", os.path.abspath("."), wait=True)
dget_resp

In [ ]:
if dget_resp[0].task[0].status == 3:
    print("Success! Downloaded a test file to your location." "Removing the file now")
    os.remove("35437908.md5sum")
else:
    if dget_resp[0].task[0].msg == "globus connect offline":
        print(
            "You need to activate your Globus Endpoint and/or ensure "
            "Globus Connect Personal is running.\n"
            "Please visit https://globus.org to activate your Endpoint"
        )
    elif dget_resp[0].task[0].msg == "permission denied":
        print(
            "Globus does not have write access to this directory.\n"
            "If you are using Globus Connect Personal, ensure that"
            "this notebook runs within one of the directories where "
            "Globus has write access. You may consider moving this"
            "notebook to a valid directory or add this directory to "
            "the Globus Connect Personal settings"
        )
    else:
        NotImplementedError(
            "Get in touch with us or consider looking online to "
            "find a solution to this problem:\n"
            f"{dget_resp[0].task[0].msg}"
        )

### (Optional) for Windows - test for Admin priveleges
Admin priveleges may be necessary to remove files or directories using Python:

In [ ]:
try:
    is_admin = os.getuid() == 0
except AttributeError:
    is_admin = ctypes.windll.shell32.IsUserAnAdmin() != 0

is_admin